In [1]:
import pandas as pd
import os
import librosa
import random

import numpy as np
import scipy
from scipy import signal

import IPython.display as ipd
import itertools

In [ ]:
df = pd.DataFrame(columns=['path', 'file'])

path =r'C:\Users\ingri\Documents\projeto ML\musan'

count=0
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.wav'):
#             print(root)
#             print(file)
            
            audio_file=os.path.join(root,file)
            df.loc[count,:]=[audio_file, file]
            
            count+=1
            
df[['class', 'filename']] = df['file'].str.split('-', 1, expand=True)

            
df.head()
    

,path,file,class,filename
0,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0000.wav,music,fma-0000.wav
1,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0001.wav,music,fma-0001.wav
2,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0002.wav,music,fma-0002.wav
3,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0003.wav,music,fma-0003.wav
4,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0004.wav,music,fma-0004.wav


In [ ]:
def features(audio_file, window_size=30, sr=16000, random_piece_duration=30):

    audio, sr = librosa.load(audio_file, sr=sr)
    duration = len(audio)/sr
#     print(duration)
    
    begin_random = random.randint(0, int(duration)-30)
    end_random = int(begin_random + random_piece_duration*sr)
    
    random_piece= audio[begin_random:int(end_random+1)]
    
    x=random_piece
    
    
    window = np.hanning(window_size)
    
    feature1 = librosa.feature.spectral_centroid(x)[0,:]
    feature1_freq = scipy.signal.convolve(window, feature1)

    feature2 = librosa.feature.spectral_flatness(x)[0,:]
    feature2_freq = scipy.signal.convolve(window, feature2)

    feature3 = librosa.feature.rms(x)[0,:]
    feature3_freq = scipy.signal.convolve(window, feature3)
    

    
    zero_crossings = librosa.zero_crossings(x, pad=False)
    feature4_time = zero_crossings.sum()

    feature5 = librosa.yin(x, 80, 1000)#[0,:]
    feature5_freq = scipy.signal.convolve(window, feature5)
    
    
    return np.array([duration, begin_random,(end_random+1), 
                     np.mean(feature1_freq), np.mean(feature2_freq), np.mean(feature3_freq), np.mean(feature5_freq),
                    np.std(feature1_freq), np.std(feature2_freq), np.std(feature3_freq), np.std(feature5_freq),
#                      np.min(feature1_freq), np.min(feature2_freq), np.min(feature3_freq), np.min(feature5_freq),
                     np.max(feature1_freq), np.max(feature2_freq), np.max(feature3_freq), np.max(feature5_freq),
                    feature4_time])


In [ ]:
for count in range(len(df)):
    try:
    
        audio_file=df.loc[count,'path']
        features_ = features(audio_file)

        df.loc[count,['duration', 'begin', 'end', 
                      'centroid_mean', 'flatness_mean', 'rms_mean', 'fund_freq_mean',
                      'centroid_std', 'flatness_std', 'rms_std', 'fund_freq_std',
#                       'centroid_min', 'flatness_min', 'rms_min', 'fund_freq_min',
                      'centroid_max', 'flatness_max', 'rms_max', 'fund_freq_max',
                      'zcr_count']] = features_
    except:
        pass


df.head()

,path,file,class,filename,duration,begin,end,centroid_mean,flatness_mean,rms_mean,fund_freq_mean,centroid_std,flatness_std,rms_std,fund_freq_std,centroid_max,flatness_max,rms_max,fund_freq_max,zcr_count
0,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0000.wav,music,fma-0000.wav,297.978750,123.0,480124.0,25848.020068,0.109002,1.402481,2687.662987,12037.467331,0.264355,0.839841,1955.128381,64605.468066,1.612153,3.371817,11699.712274,45955.0
1,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0001.wav,music,fma-0001.wav,218.044062,66.0,480067.0,22525.358942,0.544033,3.034882,3183.944030,14991.222483,1.892061,1.321824,2944.592373,80539.383943,8.193343,6.002006,14532.954545,35685.0
2,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0002.wav,music,fma-0002.wav,249.547750,50.0,480051.0,15876.487616,0.404381,2.466099,2144.403865,14852.064893,1.639711,1.655887,2695.096473,80630.350057,8.242509,6.614883,14532.954545,32784.0
3,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0003.wav,music,fma-0003.wav,324.780438,231.0,480232.0,16448.940510,0.545969,2.040806,3282.365271,17122.342654,1.929499,1.448228,3179.791033,80195.432533,8.217067,4.834601,14532.954545,33593.0
4,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0004.wav,music,fma-0004.wav,322.298750,34.0,480035.0,23781.072180,0.802061,1.366444,3653.083777,18573.922703,2.313313,0.754391,3553.434843,80647.054738,8.188816,3.076119,14532.954545,35232.0


In [3]:
df=df[df['class']!='noise'].copy()
df=pd.read_csv('musan_features.csv')
df

,Unnamed: 0,path,file,class,filename,duration,begin,end,centroid_mean,flatness_mean,...,fund_freq_mean,centroid_std,flatness_std,rms_std,fund_freq_std,centroid_max,flatness_max,rms_max,fund_freq_max,zcr_count
0,0,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0000.wav,music,fma-0000.wav,297.978750,123.0,480124.0,25848.020068,0.109002,...,2687.662987,12037.467331,0.264355,0.839841,1955.128381,64605.468066,1.612153,3.371817,11699.712274,45955.0
1,1,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0001.wav,music,fma-0001.wav,218.044062,66.0,480067.0,22525.358942,0.544033,...,3183.944030,14991.222483,1.892061,1.321824,2944.592373,80539.383943,8.193343,6.002006,14532.954545,35685.0
2,2,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0002.wav,music,fma-0002.wav,249.547750,50.0,480051.0,15876.487616,0.404381,...,2144.403865,14852.064893,1.639711,1.655887,2695.096473,80630.350057,8.242509,6.614883,14532.954545,32784.0
3,3,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0003.wav,music,fma-0003.wav,324.780438,231.0,480232.0,16448.940510,0.545969,...,3282.365271,17122.342654,1.929499,1.448228,3179.791033,80195.432533,8.217067,4.834601,14532.954545,33593.0
4,4,C:\Users\ingri\Documents\projeto ML\musan\musa...,music-fma-0004.wav,music,fma-0004.wav,322.298750,34.0,480035.0,23781.072180,0.802061,...,3653.083777,18573.922703,2.313313,0.754391,3553.434843,80647.054738,8.188816,3.076119,14532.954545,35232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,2011,C:\Users\ingri\Documents\projeto ML\musan\musa...,speech-us-gov-0248.wav,speech,us-gov-0248.wav,576.287000,152.0,480153.0,20289.067899,0.002278,...,2785.584809,4643.881540,0.003304,1.032874,677.078869,30941.723833,0.012154,4.414697,4677.082503,43021.0
1082,2012,C:\Users\ingri\Documents\projeto ML\musan\musa...,speech-us-gov-0249.wav,speech,us-gov-0249.wav,601.835125,463.0,480464.0,18975.827655,0.001887,...,3209.067580,5749.164669,0.003171,1.070322,1048.311024,36339.740529,0.018694,5.041529,9641.824736,42941.0
1083,2013,C:\Users\ingri\Documents\projeto ML\musan\musa...,speech-us-gov-0250.wav,speech,us-gov-0250.wav,126.119125,17.0,480018.0,17500.957811,0.002434,...,3559.216979,4306.291846,0.004047,1.282622,899.128143,27583.360668,0.020728,5.271114,6167.650960,35817.0
1084,2014,C:\Users\ingri\Documents\projeto ML\musan\musa...,speech-us-gov-0251.wav,speech,us-gov-0251.wav,600.032000,538.0,480539.0,20664.199703,0.002267,...,3568.270789,5606.391988,0.003078,0.541052,823.067329,39791.191050,0.015220,2.760707,6523.950774,44383.0
